# IoT-Based Smart Contract Experiment

In [1]:
#Import required libraries
from web3 import Web3
import random
import json
import matplotlib.pyplot as plt
from prettytable import PrettyTable
import math

### Function to calculate and return gas value

In [2]:
def gas_consumption(**kwargs):    
    if kwargs['solo'] == True:
        transaction_hash = kwargs['transaction']
        receipt = web3.eth.wait_for_transaction_receipt(transaction_hash).gasUsed
        return receipt

    else:
        transaction_hash = kwargs['transaction']
        receipt = web3.eth.wait_for_transaction_receipt(transaction_hash).gasUsed
        return receipt

### Hardhat Node and Contract Deployment
Run the node: <code> npx hardhat node </code>

In new terminal tab run contract: <code>npx hardhat run --network localhost scripts/script_IoT_Monitoring.js</code>
<br>*Make sure that you are in the right folder*

In [3]:
# Started HTTP and WebSocket JSON-RPC server at ...
try:
    HTTP_Server = 'http://127.0.0.1:8545'
    web3 = Web3(Web3.HTTPProvider(HTTP_Server))
    print("Connected to node:",web3.is_connected())
except:
    print('HTTP server not connected!!')

Connected to node: True


### Use-Case
<p>When transporting cheese, it's important to keep it in well-ventilated areas. This is especially crucial on longer trips through tropical regions, where cool storage is necessary. Cheese should be loaded in both new and old condition. Poor ventilation can lead to mould growth, so it's important to control the ventilation carefully. However, be cautious of excessive draughts, which can also cause damage.</p>

<p>We assume a supply chain route from New York to California where a New York based company produces cheese and through the supply route across the USA delivers the cream cheese package to the destination at California.</p>

### Use case actors
- __Contract Owner__: Granting/Revoking access of producer to create packages. 
- __Producer__: Creating/preparing cheese shipment to destination.
- __Transport__: Cargo vehicle integrated with IoT device which checks the package's conditions
- __Receiver__: Recipient of the package at the destination

In [4]:
# Asserting that we have correct number of accounts
accounts_list = web3.eth.accounts

In [5]:
#accounts_list[0]: Contract_Owner
Contract_Owner = accounts_list[0]
print(f'Contract_Owner: {Contract_Owner}')

Contract_Owner: 0xf39Fd6e51aad88F6F4ce6aB8827279cffFb92266


In [6]:
#accounts_list[1]: Producer
Producer = accounts_list[1]
print(f'Producer: {Producer}')

Producer: 0x70997970C51812dc3A010C7d01b50e0d17dc79C8


In [7]:
#accounts_list[2]: Transport
Transport = accounts_list[2]
print(f'Transport: {Transport}')

Transport: 0x3C44CdDdB6a900fa2b585dd299e03d12FA4293BC


In [8]:
#accounts_list[3]: Receiver
Receiver = accounts_list[3]
print(f'Receiver: {Receiver}')

Receiver: 0x90F79bf6EB2c4f870365E785982E1f101E93b906


<p>In previous step we deployed Smart Contract and we not the gas used</p>

In [9]:
#This might be different, When you will run deploy the contract, it will print contract address and gas used
gas_used_contract_deploy = 2048898 

#### Creating Contract Instance

In [10]:
# creating instance of the the smart contract
# Getting ABI
with open('artifacts/contracts/IoT_Monitoring.sol/InternetOfThingsContractMonitoring.json') as f:
    contract_abi = json.load(f)['abi']

In [11]:
#getting Deployed Address... NOTE THIS MIGHT CHANGE, SO MAKE SURE YOU ARE RUNNING CORRECT ADDRESS
contract_address = '0xe7f1725E7734CE288F8367e1Bb143E90bb3F0512'

In [12]:
iot = web3.eth.contract(address=contract_address, abi = contract_abi)

## Workflow
_Note_: We have make signigicant tests using testcases to check the security of the smart contract. This simulation is ONLY for checking the performance. For more detail refer to the documentations in github followed by the testcases.

### Setting Temperature Zones
<p> Setting the IoT Device's Conditions assuming it to be a refrigerator, where the temperature should be (avg) between from 1.7°C to 4.4°C or (35°F - 40°F) </p>

In [13]:
_ = iot.functions.setZones(35,40).transact({'from':Contract_Owner}).hex()
temperature_set_gas = gas_consumption(transaction = _, solo = False)

In [14]:
print('Transaction Hash:',_)
print('gas:',temperature_set_gas,'Wei')

Transaction Hash: 0x64861b720880af8841aec4593a561680819dd497e159346d2b4654022bb7a87e
gas: 68448 Wei


### Granting Access
<p> The access is granted by the contract owner to the producer to produce the package.</p>

In [15]:
_ = iot.functions.grantAccess(Producer).transact({'from':Contract_Owner}).hex()
gas_grant_access = gas_consumption(transaction = _, solo = False)

In [16]:
print('Transaction Hash:',_)
print('gas:',gas_grant_access,'Wei')

Transaction Hash: 0xd69a2d5ec177052f5851f59d24043f5a5b9cf21e5433186835526473ee2123a3
gas: 46749 Wei


### Producing Package
<p>Once the producer is granted access, the package can be produced by passing the following parameters:</p>

- __Destination__: California
- __Recipient__: Recipient based in California
- __Transport__: Transport company's address

In [17]:
_ = iot.functions.initializeProduct("California",Receiver,Transport).transact({'from':Producer})
gas_comp_package_produce = gas_consumption(transaction = _, solo = False)

In [18]:
print('Transaction Hash:',_.hex())
print('gas:',gas_comp_package_produce,'Wei')

Transaction Hash: 0xc70518a0a7a4159d5ab012fa0fbe9849af5f2f85fbb8258689311e4a6e124b3c
gas: 143482 Wei


### Transportation of Package
<p>The package is created with package Id: 1. Now it will be transported from New York to California. It takes apprimately 43 hours of continous driving without stops and approximately 6-7 days with stops. So we assume that the time frame of the journey is 6 days.</p>

<p>we assume that the temperature is well maintained and the system updates once in 24hrs (Updating with maximum temperature it reached). We will make random temperature from 30-42 degrees (<i>Note: for simplicity, we take random numbers between 30-42)</i></p>

In [19]:
Package_id = 1

In [20]:
gas_transport = []
for i in range(7):
    iot_reading = (random.randint(30, 42))
    _ = iot.functions.productMovement(Package_id,iot_reading).transact({'from':Transport})
    gas_comp = gas_consumption(transaction = _, solo = False)
    gas_transport.append(gas_comp)
    # Increase time of EVM by 24 hrs
    web3.provider.make_request("evm_increaseTime", [24 * 3600])


In [21]:
gas_transport

[189787, 141287, 141287, 141277, 141287, 141277, 139160]

### Receiving the Package
<p>Receiving the package by the recpient upon receival when received</p>

In [22]:
_ = iot.functions.receiveProduct(Package_id).transact({'from':Receiver})
gas_comp_package_receive = gas_consumption(transaction = _, solo = False)

In [23]:
print('Transaction Hash:',_.hex())
print('gas:',gas_comp_package_receive,'Wei')

Transaction Hash: 0xfb1616fbc8a262d76b8b2ed5a219c32b59fb773da0688015a07edae21d0b6b3a
gas: 29460 Wei


### Checking Product's information
<p>Asserting that first and last entry of the product's journey to be approx ~6 days</p>

In [24]:
Product_journey = iot.functions.getProductionJourney(1).call()
day_one = Product_journey[0][1]
day_seven = Product_journey[-1][1]

print("Approx time difference ~", (day_seven - day_one)/(24 * 3600))

Approx time difference ~ 6.000011574074074


### Revoking Access
<p> The access is revoked by the contract owner to the producer to stop package production.</p>

In [25]:
_ = iot.functions.revokeAccess(Producer).transact({'from':Contract_Owner}).hex()
gas_grant_revoke = gas_consumption(transaction = _, solo = False)

In [26]:
print('Transaction Hash:',_)
print('gas:',gas_grant_revoke,'Wei')

Transaction Hash: 0xc0147f127670cb5826bba06c0faa6de54f5683aef8af760300bad2877a38cc49
gas: 24826 Wei


### Results

<p>Calculating Tge Environmental setup:</p>

In [27]:
gas_block_limit = 3 * 10e6

In [28]:
#Gas consumption in workflow: Produce -> Transfer -> Receive
wf = PrettyTable()
wf.field_names = ["Action", "Executer", "Gas Used"]
wf.add_row(["Produce", "Producer", gas_comp_package_produce])
wf.add_row(["Track", "Transport", sum(gas_transport)])
wf.add_row(["Receive", "Recpient", gas_comp_package_receive])
wf

Action,Executer,Gas Used
Produce,Producer,143482
Track,Transport,1035362
Receive,Recpient,29460


In [29]:
Total_workflow_gas = gas_comp_package_produce + sum(gas_transport) + gas_comp_package_receive
number_of_actions_in_a_block = gas_block_limit/Total_workflow_gas
print(f"""
Number of such supply chain actions that can fit in 1 Ethereum block: {number_of_actions_in_a_block}
i.e. {math.floor(number_of_actions_in_a_block)} actions""",)


Number of such supply chain actions that can fit in 1 Ethereum block: 24.828188932586503
i.e. 24 actions


### Setup
As per this [article](https://www.ibisworld.com/industry-statistics/number-of-businesses/cheese-production-united-states/), There are 436 Cheese Production businesses in the US as of 2023, an increase of 0.9% from 2022.
In 2020, there were 408 business followed by 417, 433, 436 in year 2021, 2022 and 2023 respectively. 
So, we will first register (grant access) the 417 wallets and take average of gas consumptions.

In [30]:
# Take grant and revoke for full USA based on data
# Getting 400 accounts from HardHat
gas_registration = []

In [31]:
for i in range(11,428):
    try:
        business_wallet = accounts_list[i]
        _ = iot.functions.grantAccess(business_wallet).transact({'from':Contract_Owner}).hex()
        gas_grant_access = gas_consumption(transaction = _, solo = False)
        gas_registration.append(gas_grant_access)
    except:
        pass

In [32]:
assert(len(gas_registration) == 417)

In [33]:
average_gas_registration = sum(gas_registration)/len(gas_registration)
print('Average gas to register company', average_gas_registration)

Average gas to register company 46748.28057553957


In [34]:
number_of_registrations = gas_block_limit/average_gas_registration

In [35]:
print(f"""
1 Ethereum block can fit: {number_of_registrations} registrations.
i.e. {math.floor(number_of_registrations)} registrations""",)



1 Ethereum block can fit: 641.7348323971751 registrations.
i.e. 641 registrations


Assuming that 50% of business decide not to continue, therefore need to be deregistered. For simplicity we deregister alternate wallets.

In [36]:
gas_deregistration = []

In [37]:
for i in range(11,428):   
    try:
        business_wallet = accounts_list[i]
        _ = iot.functions.revokeAccess(business_wallet).transact({'from':Contract_Owner}).hex()
        gas_revoke_access = gas_consumption(transaction = _, solo = False)
        gas_deregistration.append(gas_revoke_access)
    except:
        pass

In [38]:
average_gas_deregistration = sum(gas_deregistration)/len(gas_deregistration)
print('Average gas to deregister company', average_gas_deregistration)

Average gas to deregister company 24825.280575539568


In [39]:
number_of_deregistrations = gas_block_limit/average_gas_deregistration

In [40]:
print(f"""
1 Ethereum block can fit: {number_of_deregistrations} deregistrations.
i.e. {math.floor(number_of_deregistrations)} deregistrations""",)



1 Ethereum block can fit: 1208.445556484832 deregistrations.
i.e. 1208 deregistrations


In [41]:
#Gas consumption in setup: contract deploy -> set conditions -> grantAccess -> revokeAccess
setup = PrettyTable()
setup.field_names = ["Action", "Executer", "Gas Used"]
setup.add_row(["Contract Deploy", "Owner", gas_used_contract_deploy])
setup.add_row(["Set Conditions", "Owner/Admin", temperature_set_gas])
setup.add_row(["grantAccess", "Owner/Admin", math.floor(average_gas_registration)])
setup.add_row(["revokeAccess", "Owner/Admin", math.floor(average_gas_deregistration)])
setup

Action,Executer,Gas Used
Contract Deploy,Owner,2048898
Set Conditions,Owner/Admin,68448
grantAccess,Owner/Admin,46748
revokeAccess,Owner/Admin,24825


In [42]:
# Take grant and revoke for full USA based on data
# Getting 400 accounts from HardHat
gas_re_registration = []

In [43]:
for i in range(11,428):
    try:
        business_wallet = accounts_list[i]
        _ = iot.functions.grantAccess(business_wallet).transact({'from':Contract_Owner}).hex()
        gas_grant_access = gas_consumption(transaction = _, solo = False)
        gas_re_registration.append(gas_grant_access)
    except:
        pass

In [44]:
assert(len(gas_re_registration) == 417)

In [45]:
average_gas_re_registration = sum(gas_re_registration)/len(gas_re_registration)
print('Average gas to register company', average_gas_re_registration)

Average gas to register company 46748.28057553957


In [46]:
number_of_re_registrations = gas_block_limit/average_gas_re_registration

In [47]:
print(f"""
1 Ethereum block can fit: {number_of_re_registrations} registrations.
i.e. {math.floor(number_of_re_registrations)} registrations""",)



1 Ethereum block can fit: 641.7348323971751 registrations.
i.e. 641 registrations


In [48]:
gas_re_registration

[46749,
 46749,
 46749,
 46749,
 46749,
 46749,
 46749,
 46749,
 46749,
 46749,
 46749,
 46749,
 46749,
 46749,
 46737,
 46749,
 46749,
 46749,
 46749,
 46749,
 46749,
 46749,
 46749,
 46749,
 46749,
 46749,
 46749,
 46749,
 46749,
 46749,
 46749,
 46749,
 46749,
 46749,
 46749,
 46749,
 46749,
 46737,
 46749,
 46749,
 46749,
 46749,
 46749,
 46749,
 46749,
 46749,
 46749,
 46749,
 46749,
 46749,
 46749,
 46749,
 46749,
 46737,
 46749,
 46749,
 46749,
 46749,
 46749,
 46749,
 46749,
 46749,
 46749,
 46749,
 46749,
 46749,
 46749,
 46749,
 46749,
 46749,
 46737,
 46749,
 46749,
 46749,
 46749,
 46749,
 46749,
 46749,
 46749,
 46749,
 46749,
 46749,
 46749,
 46749,
 46749,
 46749,
 46749,
 46749,
 46749,
 46749,
 46749,
 46737,
 46749,
 46749,
 46749,
 46749,
 46749,
 46737,
 46749,
 46749,
 46749,
 46749,
 46749,
 46749,
 46749,
 46737,
 46749,
 46749,
 46749,
 46749,
 46749,
 46749,
 46749,
 46749,
 46749,
 46749,
 46749,
 46749,
 46749,
 46737,
 46749,
 46749,
 46737,
 46749,
 46749,
